In [ ]:
#Let's import the libraries for our project. These are the basic library for Data Analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

Folium builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the leaflet.js library. Manipulate your data in Python, then visualize it in on a Leaflet map via folium.

Folium is not available by default. So, we first need to install it before we are able to import it.


In [ ]:

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

/bin/bash: conda: command not found
Folium installed and imported!



Generating the world map is straigtforward in Folium. You simply create a Folium Map object and then you display it. What is attactive about Folium maps is that they are interactive, so you can zoom into any region of interest despite the initial zoom level. (Credits :- Cognitive Class and Labs)

In [ ]:
# define the world map
world_map = folium.Map()

# display world map
world_map

Now, let's load our dataset. I have used the dataset of 163 Countries,in which i have taken the lattitude , longitude and CORONA Statistics. 

For Corona Statistics :- https://www.worldometers.info/coronavirus/#countries

For LAtitude and Longitude :- https://developers.google.com/public-data/docs/canonical/countries_csv

In [ ]:
df_Corona = pd.read_excel("MAP.xlsx")
df_Corona.head()

,Number,Country,Total Cases,Capital,Latitude,Longitude,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/1M pop
0,41,Afghanistan,28833,Kabul,33.939110,67.709953,8764.0,19488.0,19.0,741.0
1,103,Albania,1962,Tirane,41.153332,20.168331,1134.0,784.0,10.0,682.0
2,62,Algeria,11631,Algiers,28.033886,1.659626,8324.0,2470.0,38.0,265.0
3,133,Andorra,855,Andorra la Vella,42.546245,1.601554,792.0,11.0,8.0,11066.0
4,167,Angola,176,Luanda,-11.202692,17.873887,66.0,101.0,1.0,5.0


Now let's superimpose the locations of the cases onto the map. The way to do that in Folium is to create a feature group with its own features and style and then add it to the World_map.

In [ ]:
cases = folium.map.FeatureGroup()

for Latitude, Longitude, in zip(df_Corona.Latitude, df_Corona.Longitude):
    cases.add_child(
        folium.CircleMarker(
            [Latitude, Longitude],
            radius=5, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )
world_map.add_child(cases)

Now, Let's Change the Column names for our ease of programming.

In [ ]:
df_Corona.columns =['Number','Country','Total_Cases','Capital','Latitude','Longitude','Total_Recovered','Active_Cases','Serious_Critical','Tot_Cases_per_1M_pop']
df_Corona.head()

,Number,Country,Total_Cases,Capital,Latitude,Longitude,Total_Recovered,Active_Cases,Serious_Critical,Tot_Cases_per_1M_pop
0,41,Afghanistan,28833,Kabul,33.939110,67.709953,8764.0,19488.0,19.0,741.0
1,103,Albania,1962,Tirane,41.153332,20.168331,1134.0,784.0,10.0,682.0
2,62,Algeria,11631,Algiers,28.033886,1.659626,8324.0,2470.0,38.0,265.0
3,133,Andorra,855,Andorra la Vella,42.546245,1.601554,792.0,11.0,8.0,11066.0
4,167,Angola,176,Luanda,-11.202692,17.873887,66.0,101.0,1.0,5.0


In [ ]:
#check for type of attributes
df_Corona.dtypes

Number                    int64
Country                  object
Total_Cases               int64
Capital                  object
Latitude                float64
Longitude               float64
Total_Recovered         float64
Active_Cases            float64
Serious_Critical        float64
Tot_Cases_per_1M_pop    float64
dtype: object

Now, We will divide the countries according to their "Total Cases per 1 Million". (You can choose another attribute ) First Let's Convert the type of the Attribute i.e   
--> Tot_Cases_per_1M_pop :- float64  ---> into ---> int64

In [ ]:
df_Corona["Tot_Cases_per_1M_pop"] = df_Corona["Tot_Cases_per_1M_pop"].astype("int")

Now, We will use the concept of Binning to our column.

In [ ]:
#Let's define the range of category. for 4 Categories, we have to divide it in 5 parts
Category = np.linspace(min(df_Corona["Tot_Cases_per_1M_pop"]), max(df_Corona["Tot_Cases_per_1M_pop"]), 5)
print(Category)

#Category of the country according to the cases
Category_names = ['Risk', 'Medium Risk', 'High Risk', 'Danger']

#now, take a look what we have done so far
df_Corona['Tot_Cases_per_1M_pop_binned'] = pd.cut(df_Corona['Tot_Cases_per_1M_pop'], Category, labels= Category_names, include_lowest=True )
df_Corona[['Tot_Cases_per_1M_pop','Tot_Cases_per_1M_pop_binned']].head(10)

[    0.  7779. 15558. 23337. 31116.]


,Tot_Cases_per_1M_pop,Tot_Cases_per_1M_pop_binned
0,741,Risk
1,682,Risk
2,265,Risk
3,11066,Medium Risk
4,5,Risk
5,266,Risk
6,912,Risk
7,6840,Risk
8,946,Risk
9,293,Risk


In [ ]:
df_Corona["Tot_Cases_per_1M_pop_binned"].value_counts()

Risk           155
Medium Risk      5
Danger           1
High Risk        1
Name: Tot_Cases_per_1M_pop_binned, dtype: int64

So, we have 155 countries at risk, 5 at medium risk, 1 at danger and 1 at High risk. Now, again take a look at our Dataset. (The new column of Tot_Cases_per_1M_pop_binned will be added to our datset.)

In [ ]:
df_Corona.head()

,Number,Country,Total_Cases,Capital,Latitude,Longitude,Total_Recovered,Active_Cases,Serious_Critical,Tot_Cases_per_1M_pop,Tot_Cases_per_1M_pop_binned
0,41,Afghanistan,28833,Kabul,33.939110,67.709953,8764.0,19488.0,19.0,741,Risk
1,103,Albania,1962,Tirane,41.153332,20.168331,1134.0,784.0,10.0,682,Risk
2,62,Algeria,11631,Algiers,28.033886,1.659626,8324.0,2470.0,38.0,265,Risk
3,133,Andorra,855,Andorra la Vella,42.546245,1.601554,792.0,11.0,8.0,11066,Medium Risk
4,167,Angola,176,Luanda,-11.202692,17.873887,66.0,101.0,1.0,5,Risk


In [ ]:
# instantiate a feature group for the cases in the dataframe
cases = folium.map.FeatureGroup()

# loop through the dataset and add each to the cases feature group
for Latitude, Longitude, in zip(df_Corona.Latitude, df_Corona.Longitude):
    cases.add_child(
        folium.CircleMarker(
            [Latitude, Longitude],
            radius=3, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(df_Corona.Latitude)
longitudes = list(df_Corona.Longitude)
labels = list(df_Corona.Tot_Cases_per_1M_pop_binned)

for Latitude , Longitude , label in zip(latitudes, longitudes, labels):
    folium.Marker([Latitude, Longitude], popup=label).add_to(world_map)    
    
# add cases to map
world_map.add_child(cases)

***Choropleth Maps***

A Choropleth map is a thematic map in which areas are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map, such as population density or per-capita income.

Now, let's create our own Choropleth map of the world depicting Total Cases per 1 Million population.

In order to create a Choropleth map, we need a GeoJSON file that defines the areas/boundaries of the state, county, or country that we are interested in. In our case, since we are endeavoring to create a world map, we want a GeoJSON that defines the boundaries of all world countries. For your convenience, we will be providing you with this file, so let's go ahead and download it. Let's name it world_countries.json.

"Credits :- Cognitive Class and Labs." 

In [ ]:
# download countries geojson file
!wget --quiet https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DV0101EN/labs/Data_Files/world_countries.json -O world_countries.json
    
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


Now that we have the GeoJSON file, let's create a world map, centered around [0, 0] latitude and longitude values, with an intial zoom level of 2, and using Mapbox Bright style.

In [ ]:
world_geo = r'world_countries.json' # geojson file

# create a plain world map
world_map = folium.Map(location=[0, 0], zoom_start=2, tiles='Mapbox Bright')


And now to create a Choropleth map, we will use the choropleth method with the following main parameters:

geo_data, which is the GeoJSON file.

data, which is the dataframe containing the data.

columns, which represents the columns in the dataframe that will be used to create the Choropleth map.

key_on, which is the key or variable in the GeoJSON file that contains the name of the variable of interest. To determine that, you will need to open the GeoJSON file using any text editor and note the name of the key or variable that contains the name of the countries, since the countries are our variable of interest. In this case, name is the key in the GeoJSON file that contains the name of the countries. Note that this key is case_sensitive, so you need to pass exactly as it exists in the GeoJSON file.

"Credits :- Cognitive Class and Labs."

In [ ]:
# generate choropleth map using the total cases in the world 
world_map.choropleth(
    geo_data=world_geo,
    data=df_Corona,
    columns=['Country', 'Total_Cases'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Corona Cases in World'
)

# display map
world_map

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


The Black Portion are the countries which are not included in the dataset. So, from here, we can see that where the severity of this disease is high, where it is weak or growing.

Thank You So Much.